# [Sherlock Scraping](https://sherloc.unodc.org)

Web scraching algorithm

### - Armar la funcion para iterar los paises

### - Crear un sistema de carga reanudable

In [146]:
import selenium as sl
import requests
import os
import re
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
import urllib

# List of Latin American countries to scrape

lat_ctrs=['Antigua and Barbuda','Argentina','Bahamas','Barbados','Belize','Bolivia (Plurinational State of)','Brazil','Canada','Chile','Colombia',
    'Costa Rica','Cuba','Dominican Republic','Ecuador','El Salvador','United States of America','Grenada',
    'Guatemala','Guyana','Haiti','Honduras','Jamaica','Mexico','Nicaragua','Panama','Paraguay','Dominica','Peru',
    'Saint Kitts and Nevis','Saint Vincent and the Grenadines','Saint Lucia','Suriname','Trinidad and Tobago',
    'Uruguay','Venezuela (Bolivarian Republic of)']


##########################################################################################

def search_ctrs(url: str) -> list:
    """Function to see countries are abalible.
    """
    
    driver = webdriver.Firefox()
    driver.get(url)
    sleep(15)
    if '/treaties/' in url:
        elem=driver.find_element_by_xpath('//a[@href="#country-treaty"]')
        elem.click()
    sleep(5)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    non_ctrs = soup.find_all('a',class_="cover-parent")
    driver.quit()
    ctrs=[]
    for c in non_ctrs:
        ctr =c.get("href").split('"value":"')[1].split('"}]')[0]
        ctrs.append(ctr)
    return ctrs
##########################################################################################

def spaceend(title: str) -> str:
    """Function to delete extra spaces.
    """
    oki=0
    okd=0
    while oki == 0 or okd == 0:

        # Si no hay titulos los llamo None
        if len(title) == 0 or title== ' ':
            title = 'None'
        
        if title[-1] == ' ' and okd == 0:
            title = title[:-1]
        else:
            okd = 1
            
        if title[0] == ' ' and oki == 0:
            title = title[1:]
        else:
            oki = 1

    return title
    
##########################################################################################


def extract_urls(ctr: str,op: str) -> (dict, int):
    """Function to extract titles and urls of the
    text pages.
    """
    # Create the url for different options
    
    if op == 'Legislation':
        op = "legdb"
        add = "legislation@country_label_s"
        
    elif op == 'Strategies':
        op = "strategies"
        add = "strategy.country..country.name.html_s"
        
    elif op == 'Treaties':
        op = "treaties"
        add = "countryTreatyStatus.countriesInvolved.country..country.name.html_s1"
    
    driver = webdriver.Firefox()
    
    url = 'https://sherloc.unodc.org/cld/v3/sherloc/'+op+'/search.html?lng=en#?c=%7B%22filters%22:%5B%7B%22fieldName%22:%22en%23'+add+'%22,%22value%22:%22'+ctr+'%22%7D%5D,%22sortings%22:%22%22%7D'
    
    # Try to connect, if not, try again
    try_conn = 0
    while try_conn == 0:
        try:
            driver.get(url)
            try_conn = 1
        except:
            try_conn = 0
            sleep(1)
            
    # Take the list of urls, and its compared with the number that the page say.
    # Is compared until it matches.
    urls = []
    found = 10000
    while found != len(urls):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        soup = BeautifulSoup(driver.page_source, 'lxml')
        
        # If there are nothing close the window
        if op != 'legdb':
            nothf = soup.find('div', class_="ng-hide", id="nothing-found")
            if nothf == None:
                driver.quit()
                return {},0
        
        # Search the number of urls
        try:
            found = soup.find('div', id="found-count")
            found = int(found.find(text=True).split(' ')[1])
        except:
            continue
        
        # Search the urls
        tags = soup.find_all('a',class_="cover-parent")
        
        urls=[]
        for tag in tags:
            url = 'https://sherloc.unodc.org' + tag.get("href")
            
            if 'v3' not in url: 
                urls.append(url)
    
    # Take the titles for every url
    
    if op == 'legdb':
        cuad = str(soup.find('div',class_="animated fadeIn new my-3 ng-scope"))
        titles = titles_legis(cuad)
        
    elif op == 'strategies':
        cuads = soup.find_all('div', class_="col-md-12 col-xs-12 result-title")
        titles = {}
        for index, cuad in enumerate(cuads):
            title=cuad.find(text=True).replace('  ',' ').replace('  ',' ')
            title=spaceend(title)
            title = title.replace('á','á').replace('é','é').replace('í','í').replace('ó','ó').replace('ú','ú')
            titles[title]=[{title:urls[index]}]
            
    elif op == 'treaties':
        cuad = soup.find_all('div', class_="panel-heading signatures-heading")
        titles = {}
        for index, cu in enumerate(cuad):
            text=cu.find_all(text=True)
            line1=text[0].replace('\n                            ','').replace('\n                        ','')
            
            if len(text) == 4:
                line2=text[2].replace('\xa0','')
                title = (line1+' '+line2).replace('  ',' ').replace('  ',' ')
                title=spaceend(title)
                title = title.replace('á','á').replace('é','é').replace('í','í').replace('ó','ó').replace('ú','ú')
                titles[title]=[{title:urls[index]}]
            else:
                title = line1.replace('  ',' ').replace('  ',' ')
                title=spaceend(title)
                title = title.replace('á','á').replace('é','é').replace('í','í').replace('ó','ó').replace('ú','ú')
                titles[title]=[{title:urls[index]}]
                
    driver.quit()
    
    return titles, len(urls)

##########################################################################################

def remplace_title_text(text: list) -> list:
    """Replace all the unintedible unicode characters with
    the alternative characters to latex.
    """
    
    ctr_text={'«':'$<<$','»':'$>>$',"_":"\_","%":"\%","&":"\&",
              "$":"\\\$ ","":"•","":"•","":"•",'"':"'",'ñ':'ñ',
              'ó':'ó','á':'á','é':'é','í':'í','ú':'ú','Á':'Á',
              'É':'É','Í':'Í','Ó':'Ó','Ú':'Ú','−':'-','─':'-',
              '«':'$\ll$ ','»':'$\gg$ ','ê':'ê','ô':'ô','ã':'ã',
              'õ':'õ','à':'à','è':'è','ô':'ô','ç':'ç','â':'â',
              'ü':'ü','#':'\#','':'.',' ':'•',' ':'•','':'•',
              ';':';',' ':'•','`(':'(','а':'a','ï':'ï','û':'û',
              '}-':')-','{':'(','}':')','`S':'S','`E':'E',
              '':'$\rightarrow$ ','●':'•','\\':'',"`,":"´,",'"':'\\"'}
    
    
    ctr_text = dict((re.escape(k), v) for k, v in ctr_text.items())
    pattern_text = re.compile("|".join(ctr_text.keys()))
    
    new_text=[]
    for n,line in enumerate(text):
        if text[n]=='\n' or text[n]=='\xa0':
            new_text.append('')
            new_text.append('')
        text[n] = pattern_text.sub(lambda m: ctr_text[re.escape(m.group(0))], text[n])
        new_text.append(text[n])
    
    return new_text

##########################################################################################

def load_text(url: str) -> (list, str):
    """Searh the respective text and take the urls
    to download the attached files.
    """
    
    try_conn=0
    while try_conn==0:
        try:
            res = requests.get(url)
            try_conn=1
        except:
            try_conn=0
            sleep(1)
            
    html_page = res.content
    soup = BeautifulSoup(html_page, 'lxml')
    attach = []
    
    if '/legislation/' in url:
        title, text = ('epa', ['salepa', 'quepacho'])
        cuad = soup.find('div',class_="case-law col-md-8 col-lg-9 col-xs-12 topSpace10")
        text = cuad.find_all(text=True)
        
        if 'Attachments' in text:
            for at in str(cuad.find('div',class_="attachments")).split('href="'):
                if '/cld/doc' in at:
                    attach.append('https://sherloc.unodc.org' + at.split('" ')[0])
                    
    elif '/treaties/status/' in url:
        cuad = soup.find_all('div',class_="container")
        text = cuad[3].find_all(text=True)
        if 'Attachments' in text:         # I can't find attached files for treaties
            print("Attached file not downloaded. " + title)
            
    elif '/strategies/' in url:
        cuad = soup.find('div',class_="col-md-9 col-lg-9 col-xs-12 topSpace20")
        text = cuad.find_all(text=True)
        
        if 'Attachments' in text:
            for at in str(cuad.find('div',class_="topSpace10")).split('href="'):
                if '/cld/doc' in at:
                    attach.append('https://sherloc.unodc.org' + at.split('" ')[0])
    
    text = remplace_title_text(text)
    
    return text, attach

##########################################################################################

def down_attach(url: str, path: str, fold_title: str) -> (list):
    """To diferent tipe of url download the attached file
    saving it in a dedicated folder.
    """
    
    # Cut the long names
    if len(fold_title) < 139: 
        fold_path = path + '/' + fold_title
        if not os.path.exists(fold_path):
            os.makedirs(fold_path)
    else:
        ps= fold_title[0:135]
        for i in range(len(ps)):
            if ps[-1] != ' ':
                ps = ps[:-1]
            else:
                fold_title = ps + '(...)'
    
    fold_path = path + '/' + fold_title + '/Attachments/'
    if not os.path.exists(fold_path):
        os.makedirs(fold_path)
    
    not_dw = ''
    
    # Si el link es de una pagina buscar el descargable, sino descargalo
    if '.html' in url:
        con=0
        while con == 0: # Wait to connect to internet
            try:
                res = requests.get(url)
                con = 1
            except:
                con = 0
                sleep(1)
        
        html_page = res.content
        soup = BeautifulSoup(html_page, 'lxml')
        cuad = soup.find_all('div',class_="link")
        
        if len(cuad)==0: # 404 pages
            not_dw = 'Error 404: ' + url
        elif len(cuad) > 1:
            for link in cuad:
                not_dw = not_dw + ',' + 'https://sherloc.unodc.org' + str(link).split('href="')[1].split('" ')[0]
        else:
            attach = 'https://sherloc.unodc.org' + str(cuad[0]).split('href="')[1].split('" ')[0]
            
    elif '.pdf' in url:
        attach = url
    else:
        not_dw = url

    if not_dw == '':
        title = attach.split('/')[-1].split('.pdf')[0]
        fold_path = fold_path + title
        
        try:
            r = requests.get(attach, stream=True)
            with open(fold_path+'.pdf', 'wb') as f:
                f.write(r.content)
        except:
            not_dw = url
    return not_dw

##########################################################################################

def generate_latex_text(title: str, text: list) -> (list):
    """This function generates the list with all lines that will be agregated
    to the .tex file."""
    
    title = title.replace("_", "\_").replace("&","\&").replace('"','\\"')
    
    latex_text=['\\documentclass{article}',
                '\\usepackage[utf8]{inputenc}',
                '\\title{'+ title +'}',
                '\\author{}',
                '\\date{}',
                '\\begin{document}',
                '\\maketitle']
    
    for line in text:
        latex_text.append(line)
    end_text='\\end{document}'
    latex_text.append(end_text)
    
    return latex_text

##########################################################################################

def titles_legis(cuad_str: str) -> (dict):
    """Search the titles in the legislation pages.
    """
    # Take the part with the principal titles
    principal_titles = cuad_str.split('<p class="law-title py-1 ng-binding">')
    vals = {}
    for raw_title in principal_titles:
        if '<span class="ng-binding ng-scope"' in raw_title:
            
            # Take the title
            title = raw_title[raw_title.index('</span>')+len('</span>'):raw_title.index('</p>')]
            
            title = spaceend(title[0:-33]).replace(" \xa0","").replace("&amp;", "&").replace('  ',' ').replace('  ',' ')
            title = title.replace('á','á').replace('é','é').replace('í','í').replace('ó','ó').replace('ú','ú')
            vals[title] = []
            
            # Take the part with subtitles
            raw_subtitles_split = raw_title.split('<p class="leg-article mx-2 py-2 ng-scope"')
            
            for raw_subtitles in raw_subtitles_split:
                elem = {}
                
                # Take the subtitles
                if '<span class="ng-binding ng-scope"' in raw_subtitles:
                    subtitles = raw_subtitles.split('<span class="ng-binding ng-scope"')
                    subtitle = ''
                    for raw_subtitle in subtitles:
                        if '</i></span>' in raw_subtitle:
                            subtitle = subtitle + raw_subtitle[raw_subtitle.index('</i></span>')+len('</i></span>'): raw_subtitle.index('</span><!')]
                            
                    subtitle = subtitle.replace(" \xa0","").replace("&amp;", "&")
                    subtitle = spaceend(subtitle).replace('  ',' ').replace('  ',' ')
                    subtitle = subtitle.replace('á','á').replace('é','é').replace('í','í').replace('ó','ó').replace('ú','ú')
                    
                    link = raw_subtitles[raw_subtitles.index('href="')+len('href="'): raw_subtitles.index('" ng-click="$event.stopPropagation();')]#'"></a>')]
                    elem[subtitle] = 'https://sherloc.unodc.org' + link
                    
                    vals[title].append(elem)
    return vals

##########################################################################################

def create_file_system(path: str, fold_title: str, latex_text:list, page_title: str, url:str) -> (list, str):
    """Function to create the .pdf files and his respective folder system.
    """
    
    fold_title = fold_title.replace('/','_').replace('  ',' ')
    page_title = page_title.replace('/','_').replace('  ',' ')
    
    # Cambio las comillas que dan problemas
    comillas = 0
    if '"' in fold_title or '"' in page_title:
        fold_title = fold_title.replace('"',"'''")
        page_title = page_title.replace('"',"'''")
        comillas = 1
    
    # Acorto los nombres
    if len(page_title) > 140:
        ps= page_title[0:135]
        for i in range(len(ps)):
            if ps[-1] != ' ':
                ps = ps[:-1]
            else:
                page_title = ps + '(...)'
    
    if len(fold_title) < 139:
        fold_path = path + '/' + fold_title
        if not os.path.exists(fold_path):
            os.makedirs(fold_path)
    else:
        ps= fold_title[0:135]
        for i in range(len(ps)):
            if ps[-1] != ' ':
                ps = ps[:-1]
            else:
                fold_title = ps + '(...)'
        
        fold_path = path+ '/' + fold_title
        
        
        if not os.path.exists(fold_path):
            os.makedirs(fold_path)    
    
    
    # Creo el .tex y luego el pdf
    for j,line in enumerate(latex_text):
        os.system('(echo "' + line +'" >> "'+ page_title + '.tex")')
        
    os.system('(pdflatex "' + page_title + '.tex" >/dev/null 2>&1)')
    
    # Espera a que se generen los archivos
    OK = 0
    while (OK < 10 and
           not(os.path.exists(page_title+'.aux') and
               os.path.exists(page_title+'.log') and
               os.path.exists(page_title+'.pdf'))):
        
        sleep(1)
        OK += 1
        if OK == 10:
            print('Error loading: ' + fold_title + page_title + ' '*40 +'\n'+ url +'\n')
    
    
    if OK < 10:
        Black_List = []
        msg = page_title + ' OK'
        os.system('rm "'+page_title+'.aux"')
        os.system('rm "'+page_title+'.log"')
        if comillas == 0:
            os.system('mv "'+page_title+'.pdf" "'+fold_path+'/'+page_title+'.pdf"')
        else:
            os.system('mv "'+page_title+'.pdf" "'+fold_path+'/'+page_title.replace("'''",'\\"')+'.pdf"')
        os.system('rm "'+page_title+'.tex"')
        
    elif OK == 10:
        Black_List = [url]
        msg = page_title + ' Fail'
        os.system('rm "'+page_title+'.aux"')
        os.system('rm "'+page_title+'.log"')
        os.system('mv "'+page_title+'.tex" "'+page_title+'.error.tex"')
    
    return Black_List, msg
        
##########################################################################################
    
def create_fold(ctr: str, path: str) -> (list, list, list):
    """This function creates the first three folders for each country.
    """
    
    tipes=['Legislation','Strategies','Treaties']
    init_path = path
    Black_List, attachs, not_dw, msgs = [], [], [], []
    
    print(ctr+ ': ')
    
    for tipe in tipes:
        titles, top = extract_urls(ctr,tipe)
        
        path = init_path + '/' + tipe
        
        val=0
        for fold_title,pages in titles.items():
            fold_at = []
            for page in pages:
                val+=1
                page_title,url = list(page.items())[0]
                text ,ats = load_text(url)
                latex_text = generate_latex_text(page_title,text)
                bl,msg = create_file_system(path, fold_title, latex_text, page_title, url)
                Black_List = Black_List + bl
                
                if ats != []:
                    for at in ats:
                        if at not in fold_at:
                            nd = down_attach(at, path, fold_title)
                            if nd != '':
                                not_dw.append(nd)
                            fold_at.append(at)
                            
                        if at not in attachs:
                            attachs.append(at)
                msgs.append(msg)
                
                if len(msgs) > 1:
                    print(' '*len(ctr) +' ' * len(msgs[-2])+' ' * 34,end='\r')
                print(' '*len(ctr) + '-' + tipe+ ' ' +str(int(val*100/top))+'% | '+ msgs[-1],end='\r')
        
        print(' '*len(ctr) + '-' + tipe+ ' 100%' + ' ' * (len(msgs[-1])+4))
    
    return Black_List, attachs, not_dw

##########################################################################################

def download_lat_files(path: str, lat_ctrs: list) -> (list, list, list):
    """This function take the of the pages for each country and stores them in files.
    """
    
    if not os.path.exists(path):
            os.makedirs(path)
    loaded_ctr=os.listdir(path)
    
    Black_list, attach, not_down = [], [], []
    
    for lc in lat_ctrs:
        if lc + '_inc' in loaded_ctr:
            os.system('rm -rf "'+path+lc+'_inc'+'"')
        
        if not lc in loaded_ctr:
            print('#########################################')
            
            inc_ctr = lc + '_inc'
            inc_path = path + inc_ctr
            
            bl, at, nd = create_fold(lc,inc_path)
            
            Black_list, attach, not_dw = Black_list + bl, attach + at, not_dw + nd

            # Si tenes no se descargaron todos los archivos borro la carpeta
            if bl == []:
                os.system('mv "'+path+inc_ctr+'" "'+path+lc+'"')
            else:
                os.system('rm -rf "'+path+inc_ctr+'"')
                
            print('#########################################\n\n')
            
    return Black_list, attach, not_dw


In [147]:
path='/home/kenneths/SherlockData/'
bl, at, nd = download_lat_files(path,lat_ctrs)

#########################################
Venezuela (Bolivarian Republic of): 
                                  -Legislation 100%                                                                                                                                                               
                                  -Strategies 100%                     
                                  -Treaties 100%                                                                                                                                                                
#########################################




## Individual proving

In [138]:
url='https://sherloc.unodc.org/cld//legislation/cri/ley_no._7.317_ley_de_conservacion_de_la_vida_silvestre/capitulo_iv/articulo_14-27/capitulo_iv.html?lng=en&amp;tmpl=sherloc'
page_title = 'Ley No. 7.317 Ley de Conservación de la Vida SilvestreCapitulo IV: Conservación y manejo de la vida silvestre Articulo 14-27'

fold_title='test'
path='/home/kenneths/SherlockData/test'
text,at=load_text(url)
latex_text=generate_latex_text(page_title,text)
bl,msg = create_file_system(path, fold_title, latex_text, page_title, url)